In [30]:
import pymysql
import pandas as pd
import math
import yfinance as yf
import numpy as np
from nsepy.derivatives import get_expiry_date
from nsepy import get_history
from dateutil.relativedelta import relativedelta, TH
from datetime import date, timedelta
from nsepy.derivatives import get_expiry_date
comp_list = ['ADANIPORTS', 'ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 
        'BANDHANBNK', 'BHARTIARTL','BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 
        'EICHERMOT', 'FEDERALBNK', 'GRASIM', 'HCLTECH', 'HDFC','HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
        'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'IDFCFIRSTB', 'INDUSINDBK', 'INFY', 'IOC', 'ITC', 
        'JSWSTEEL', 'KOTAKBANK', 'LT', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'PNB', 'POWERGRID', 
        'RBLBANK', 'RELIANCE', 'SBILIFE', 'SBIN', 'SHREECEM', 'SUNPHARMA','TATACONSUM', 'TATAMOTORS',
        'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'UPL', 'WIPRO']

def get_last_thurs(year,month):
    dt= date(year,month ,1)
    cmon = month 
    t=dt
    for i in range(1, 6):
        t = dt + relativedelta(weekday=TH(i))
        if t.month != cmon:
        # since t is exceeded we need last one  which we can get by subtracting -2 since it is already a Thursday.
            t = t + relativedelta(weekday=TH(-2))
            break
    return t

dict_rvol={}
for comp in comp_list:
    dict_rvol[comp]=0
    
ZZZ= []

today = date.today()
cmon = today.month
t=date.today()
for i in range(1, 6):
    t = today + relativedelta(weekday=TH(i))
    if t.month != cmon:
        # since t is exceeded we need last one  which we can get by subtracting -2 since it is already a Thursday.
        t = t + relativedelta(weekday=TH(-2))
        break
        
print('expiry on : ',end='')
print(t)
if(t.day >today.day+7 ):
    print('expiry in this month')
    
    for i in range(1,today.month):
        #getting last thurs of that month
        thurs = get_last_thurs(year=today.year,month=i)
        print('expiry on '+ str(thurs))
        
        
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year  ,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            #print(close.shape[0])
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
            close = close[-days_considered:]
#             print(days_considered)
            close = close[~np.isnan(close)]
#             print(close.shape[0])
#             print(close[-1])
            
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
#             print(len(log))
#             print('log and std')
#             print(log)
            std=np.std(log)
#             print(std)
            rvol = std* np.sqrt(252)
            dict_rvol[comp]+=rvol/12
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            
            
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
    
    
    
    for i in range(today.month,13):
        #getting last thurs of that month
#         set_thurs = get_expiry_date(year=today.year-1,month=i)
#         list_thurs = []
#         for _ in set_thurs:
#             list_thurs.append(_)
#         list_thurs.sort()
#         thurs=list_thurs[-1]
        thurs = get_last_thurs(year=today.year-1,month=i)
        print('expiry on '+ str(thurs))
        
        
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year -1  ,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            #print(close.shape[0])
#             print(close[-1])
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
            close = close[-days_considered:]
            close = close[~np.isnan(close)]
#             print(days_considered)
#             print(close.shape[0])
          
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
#             print('log and std')
#             print(log)
            std=np.std(log)
            rvol = std* np.sqrt(252)
#             print(std)
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
            

elif( t.day<today.day):
    days_to_expiry=35
    print('expiry in next month')
    for i in range(1,today.month+1):
        #getting last thurs of that month
        thurs = get_last_thurs(year=today.year,month=i)
        print('expiry on '+ str(thurs))
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            
            #print(close.shape[0])
             
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
            close = close[-days_considered:]
#             print(days_considered)
#             print(close.shape[0])
            close = close[~np.isnan(close)]
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
#             print('log and std')
#             print(log)
#             print(len(log))
            std=np.std(log)
#             print(std)
            rvol = std* np.sqrt(252)
            
            
               
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            
            
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
    
           
    
    for i in range(today.month+1,13):
        #getting last thurs of that month
        thurs = get_last_thurs(year=today.year-1,month=i)
        
        print('expiry on '+ str(thurs))
        
        
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year -1  ,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            #print(close.shape[0])
            
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
#             print(days_considered)
            close = close[-days_considered:]
            #print(days_considered)
#             print(close.shape[0])
          #np.log10(close/close.shift())
            close = close[~np.isnan(close)]
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
            
            std=np.std(log)
            rvol = std* np.sqrt(252)
            
            
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            
                
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
            

expiry on : 2022-03-31
expiry in this month
expiry on 2022-01-27
getting data from ADANIPORTS
0.019961128618496842 ADANIPORTS rvol 
getting data from ASIANPAINT
0.018457363616155537 ASIANPAINT rvol 
getting data from AUBANK
0.029532197382204903 AUBANK rvol 
getting data from AXISBANK
0.02412236850561145 AXISBANK rvol 
getting data from BAJAJ-AUTO
0.02061233317396274 BAJAJ-AUTO rvol 
getting data from BAJAJFINSV
0.02800360342829904 BAJAJFINSV rvol 
getting data from BAJFINANCE
0.025256256256769014 BAJFINANCE rvol 
getting data from BANDHANBNK
0.02284698934522429 BANDHANBNK rvol 
getting data from BHARTIARTL
0.01995827664902285 BHARTIARTL rvol 
getting data from BPCL
0.016523110416392497 BPCL rvol 
getting data from BRITANNIA


KeyboardInterrupt: 

In [24]:
print(dict_rvol)

{'ADANIPORTS': 0.17732100783704988, 'ASIANPAINT': 0.12279651764222473, 'AUBANK': 0.19636290776743007, 'AXISBANK': 0.12851147937637314, 'BAJAJ-AUTO': 0.11637187648354819, 'BAJAJFINSV': 0.16165321439281685, 'BAJFINANCE': 0.1602820356051033, 'BANDHANBNK': 0.1695672699590855, 'BHARTIARTL': 0.1217762335823667, 'BPCL': 0.12271894149800208, 'BRITANNIA': 0.0919425395353547, 'CIPLA': 0.1081697016899526, 'COALINDIA': 0.15089614301418464, 'DIVISLAB': 0.12150058467924936, 'DRREDDY': 0.10647201111077245, 'EICHERMOT': 0.12762244218791846, 'FEDERALBNK': 0.1690738214715094, 'GRASIM': 0.13562788298784278, 'HCLTECH': 0.12308818445096797, 'HDFC': 0.12162523354078969, 'HDFCBANK': 0.10482670839166387, 'HDFCLIFE': 0.10827188227415156, 'HEROMOTOCO': 0.11148571050343038, 'HINDALCO': 0.17615380798165733, 'HINDUNILVR': 0.09551389706348137, 'ICICIBANK': 0.12199650268573597, 'IDFCFIRSTB': 0.17923898217381518, 'INDUSINDBK': 0.1704161521533902, 'INFY': 0.10414934168756024, 'IOC': 0.12309698107129928, 'ITC': 0.11550

In [29]:
df = pd.DataFrame(list(dict_rvol.items()),columns = ['Symbol', 'Hist_RV'])
df['Hist_RV'] = round(df['Hist_RV'] * 100,2)
df.head(100)

,Symbol,Hist_RV
0,ADANIPORTS,17.73
1,ASIANPAINT,12.28
2,AUBANK,19.64
3,AXISBANK,12.85
4,BAJAJ-AUTO,11.64
5,BAJAJFINSV,16.17
6,BAJFINANCE,16.03
7,BANDHANBNK,16.96
8,BHARTIARTL,12.18
9,BPCL,12.27
